![Rijksoverheid logo](https://www.rijksoverheid.nl/binaries/content/gallery/rijksoverheid/channel-afbeeldingen/logos/logo-ro.svg)

# Dutch Government Policy QA dataset
This dataset is open-source and can be found on the open data portal of the [Rijksoverheid](https://www.rijksoverheid.nl/opendata/vac-s). It contains up to 2500 frequently asked questions of Dutch citizens. The questions are concerned with Dutch government policies and contain topics like "Belasting", "Asbest", or "Klimaat".<br>
More info about the status and contact information can be found [here](https://data.overheid.nl/dataset/vraag-antwoordcombinaties-van-rijksoverheid-nl#panel-description). <br><br>

**How to use:** <br>
It is best to use Google Colab and run the notebook to get results

### In this notebook:
- The Dutch policy QA data is imported via api with a crawler
- Initial EDA is performed to check the size, completeness, and volume
- The neccessary columns are exported as a csv
- A short answer is retrieved manually from the context
- Extra EDA is performed with on the final dataset
- The PolicyQA dataset is converted to the correct input for the QA model by using our DF to JSON converter
- The PolicyQA dataset in JSON format is used as input for the model

In [ ]:
# if using Colab, install necessary libraries
%pip install transformers

In [4]:
# Import libraries
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import json
import time
import matplotlib as plt

## Import Data
First the data is imported using crawler.py <br>
Then the data is checked for volume, completeness etc.

In [ ]:
# Run crawler
!python3 /scripts/crawler.py

In [6]:
pwd!

'/Users/berry/Code/domain-adaptation-transformers-forQA/notebooks'

In [7]:
# Import csv
df = pd.read_csv('../data/policyqa-raw.csv')
df.head()

,Unnamed: 0,id,type,canonical,dataurl,question,introduction,content,authorities,subjects,themes,organisationalunits,lastmodified,license,rightsholders,language,location
0,0,b98b7797-7d40-43a7-bc6d-bf7f96d31dd0,vraag en antwoord,https://www.rijksoverheid.nl/onderwerpen/inkom...,https://opendata.rijksoverheid.nl/v1/sources/r...,Hoe lang moet ik mijn financiële administratie...,<p>Voor particulieren geldt geen wettelijke be...,[{'paragraphtitle': 'Geen bewaarplicht voor pa...,['Ministerie van Financiën'],"['Inkomstenbelasting', 'Zelfstandigen zonder p...","['Economie', 'Belastingen, uitkeringen en toes...",['Ministerie van Financiën'],2021-09-06T12:18:10.872Z,CC0 1.0 Universal,['Ministerie van Algemene Zaken'],nl-NL,Nederland
1,0,a59a7ef3-8c7c-4d38-9a6a-142a9fca6eec,vraag en antwoord,https://www.rijksoverheid.nl/onderwerpen/belas...,https://opendata.rijksoverheid.nl/v1/sources/r...,Wat is een belastingverdrag?,<p>Een belastingverdrag is een overeenkomst tu...,[{'paragraphtitle': 'Toepassen van het belasti...,['Ministerie van Financiën'],['Belastingverdragen'],"['Belastingen, uitkeringen en toeslagen', 'Int...",['Ministerie van Financiën'],2020-06-22T13:12:15.045Z,CC0 1.0 Universal,['Ministerie van Algemene Zaken'],nl-NL,Nederland
2,0,193accef-6ff6-4a05-a89c-b6ff37389745,vraag en antwoord,https://www.rijksoverheid.nl/onderwerpen/paspo...,https://opendata.rijksoverheid.nl/v1/sources/r...,Is mijn beschadigde paspoort of identiteitskaa...,<p>Zijn de echtheidskenmerken van uw paspoort ...,[{'paragraphtitle': 'Nieuw reisdocument aanvra...,['Ministerie van Binnenlandse Zaken en Koninkr...,['Paspoort en identiteitskaart'],"['Migratie en reizen', 'Recht, veiligheid en d...",['Ministerie van Binnenlandse Zaken en Koninkr...,2018-01-03T10:05:53.606Z,CC0 1.0 Universal,['Ministerie van Algemene Zaken'],nl-NL,Nederland
3,0,913baae5-4e13-4270-bab2-d8035a15f032,vraag en antwoord,https://www.rijksoverheid.nl/onderwerpen/besch...,https://opendata.rijksoverheid.nl/v1/sources/r...,Moet een bank een kopie van mijn paspoort make...,<p>Banken en financiële dienstverleners zijn n...,[{'paragraphtitle': 'Financiële instelling doe...,['Ministerie van Financiën'],"['Bescherming van consumenten', 'Financiële se...","['Economie', 'Recht, veiligheid en defensie']",['Ministerie van Financiën'],2022-02-04T12:48:50.668Z,CC0 1.0 Universal,['Ministerie van Algemene Zaken'],nl-NL,Nederland
4,0,1c255df3-9d57-4d26-a4d9-7637b54d47e6,vraag en antwoord,https://www.rijksoverheid.nl/onderwerpen/geldz...,https://opendata.rijksoverheid.nl/v1/sources/r...,Waarom is de euro ingevoerd?,<p>Er zijn een aantal redenen voor de invoerin...,[{'paragraphtitle': 'Einde aan schommelende wi...,['Ministerie van Financiën'],"['Geldzaken', 'Financiën Europese Unie']","['Economie', 'Internationale samenwerking']",['Ministerie van Financiën'],2021-08-24T08:34:50.465Z,CC0 1.0 Universal,['Ministerie van Algemene Zaken'],nl-NL,Nederland


## Initial EDA

In [8]:
# Check info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1988 entries, 0 to 1987
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           1988 non-null   int64 
 1   id                   1988 non-null   object
 2   type                 1988 non-null   object
 3   canonical            1988 non-null   object
 4   dataurl              1988 non-null   object
 5   question             1988 non-null   object
 6   introduction         1981 non-null   object
 7   content              1917 non-null   object
 8   authorities          1988 non-null   object
 9   subjects             1978 non-null   object
 10  themes               1978 non-null   object
 11  organisationalunits  1988 non-null   object
 12  lastmodified         1988 non-null   object
 13  license              1988 non-null   object
 14  rightsholders        1988 non-null   object
 15  language             1988 non-null   object
 16  locati

In [ ]:
# Check unique themes and organizations
df['organisationalunits'].unique()

## Clean and export Data
We remove HTML tags as well as brackets. Brackets needed to be removed in order to create the input file as JSON.<br>
The data is exported so we can annotate the short answer manually.

In [ ]:
# Take first 7 columns
df = df.iloc[:, 0:7]
# Remove column 3 and 4
df = df.drop(columns=["canonical", "dataurl"])
df

In [ ]:
df.to_csv('policyqa-raw.csv', encoding = 'utf-8-sig') 